## Script

In [ ]:
import pandas as pd
import os
from io import StringIO
import xml.etree.ElementTree as ET


In [ ]:
# Versões dos pacotes usados neste jupyter notebook
#%reload_ext watermark
#%watermark -a "Danimar Grando"

In [ ]:
# Namespace do XML
ns = {'nfe': 'http://www.portalfiscal.inf.br/nfe'}

In [ ]:
# Carrega dados do diretório
diretorio = 'arquivos_xml'

In [ ]:
# Cria uma lista vazia para posteriormente converter em Dataframe para armazenar os dados
dataframes = []

In [ ]:
# Iterar pelos arquivos no diretório
for arquivo in os.listdir(diretorio):
    if arquivo.endswith(".xml"):  # Filtra apenas arquivos XML
        caminho_completo = os.path.join(diretorio, arquivo)

        # Lendo os arquivos
        tree = ET.parse(caminho_completo)
        root = tree.getroot()

        # Inicializar variáveis gerais da nota fiscal
        chave_acesso = numero_nota = nat_op = data_emissao = None
        emitente_nome = cnpj_emitente = uf_emitente = destinatario_nome = cnpj_destinatario = uf_destinatario = None

        # Capturar a chave de acesso
        protNFe = root.find('.//nfe:protNFe/nfe:infProt', ns)
        if protNFe is not None:
            chave_acesso = protNFe.find('nfe:chNFe', ns).text if protNFe.find('nfe:chNFe', ns) is not None else None

        # Capturar os dados gerais da NF
        ide = root.find('.//nfe:infNFe/nfe:ide', ns)
        if ide is not None:
            numero_nota = ide.find('nfe:nNF', ns).text if ide.find('nfe:nNF', ns) is not None else None
            nat_op = ide.find('nfe:natOp', ns).text if ide.find('nfe:natOp', ns) is not None else None
            data_emissao = ide.find('nfe:dhEmi', ns).text if ide.find('nfe:dhEmi', ns) is not None else None

        # Capturar os dados do emitente
        emit = root.find('.//nfe:infNFe/nfe:emit', ns)
        if emit is not None:
            emitente_nome = emit.find('nfe:xNome', ns).text if emit.find('nfe:xNome', ns) is not None else None
            cnpj_emitente = emit.find('nfe:CNPJ', ns).text if emit.find('nfe:CNPJ', ns) is not None else None
        
        # Dados do endereço do emitente
        enderEmit = root.find('.//nfe:emit/nfe:enderEmit', ns)
        if enderEmit is not None:
            uf_emitente = enderEmit.find('nfe:UF', ns).text if enderEmit.find('nfe:UF', ns) is not None else None   
        

        # Capturar os dados do destinatário
        dest = root.find('.//nfe:infNFe/nfe:dest', ns)
        if dest is not None:
            destinatario_nome = dest.find('nfe:xNome', ns).text if dest.find('nfe:xNome', ns) is not None else None
            cnpj_destinatario = dest.find('nfe:CNPJ', ns).text if dest.find('nfe:CNPJ', ns) is not None else None
            
        # Dados do endereço do emitente
        enderDest = root.find('.//nfe:dest/nfe:enderDest', ns)
        if enderDest is not None:
            uf_destinatario = enderDest.find('nfe:UF', ns).text if enderDest.find('nfe:UF', ns) is not None else None  

        # Capturar os dados de cada produto
        for det in root.findall('.//nfe:det', ns):
            produtos = det.find('nfe:prod', ns)
            if produtos is not None:
                cod_ean = produtos.find('nfe:cEAN', ns).text if produtos.find('nfe:cEAN', ns) is not None else None
                nome_produto = produtos.find('nfe:xProd', ns).text if produtos.find('nfe:xProd', ns) is not None else None
                ncm = produtos.find('nfe:NCM', ns).text if produtos.find('nfe:NCM', ns) is not None else None
                cest = produtos.find('nfe:CEST', ns).text if produtos.find('nfe:CEST', ns) is not None else None
                cfop = produtos.find('nfe:CFOP', ns).text if produtos.find('nfe:CFOP', ns) is not None else None
                vprod = produtos.find('nfe:vProd', ns).text if produtos.find('nfe:vProd', ns) is not None else None
                vdesc = produtos.find('nfe:vDesc', ns).text if produtos.find('nfe:vDesc', ns) is not None else None

                # Capturar os dados dos impostos (ICMS e PIS)
                imposto = det.find('nfe:imposto', ns)
                cst_icms = origem_prod = vBC_icms = pICMS = vICMS = None
                if imposto is not None:
                    icms00 = imposto.find('nfe:ICMS/nfe:ICMS00', ns)
                    if icms00 is not None:
                        origem_prod = icms00.find('nfe:orig', ns).text if icms00.find('nfe:orig', ns) is not None else None
                        cst_icms = icms00.find('nfe:CST', ns).text if icms00.find('nfe:CST', ns) is not None else None
                        vBC_icms = icms00.find('nfe:vBC', ns).text if icms00.find('nfe:vBC', ns) is not None else None
                        pICMS = icms00.find('nfe:pICMS', ns).text if icms00.find('nfe:pICMS', ns) is not None else None
                        vICMS = icms00.find('nfe:vICMS', ns).text if icms00.find('nfe:vICMS', ns) is not None else None

                cst_icms10 = origem_prod10 = vBC_icms10 = pICMS10 = vICMS10 = vBC_icms_ST = pICMS_ST = vICMS_ST = None

                if imposto is not None:
                    icms10 = imposto.find('nfe:ICMS/nfe:ICMS10', ns) if imposto is not None else None
                    if icms10 is not None:
                        origem_prod10 = icms10.find('nfe:orig', ns).text if icms10 is not None else None
                        cst_icms10 = icms10.find('nfe:CST', ns).text if icms10 is not None else None
                        vBC_icms10 = icms10.find('nfe:vBC', ns).text if icms10 is not None else None
                        pICMS10 = icms10.find('nfe:pICMS', ns).text if icms10 is not None else None
                        vICMS10 = icms10.find('nfe:vICMS', ns).text if icms10 is not None else None
                        vBC_icms_ST = icms10.find('nfe:vBC', ns).text if icms10 is not None else None
                        pICMS_ST = icms10.find('nfe:pICMS', ns).text if icms10 is not None else None
                        vICMS_ST = icms10.find('nfe:vICMS', ns).text if icms10 is not None else None

                origem_prod61 = cst_icms61 = vBC_icms61 = pICMS61 = vICMS61 = None

                if imposto is not None:
                    icms61 = imposto.find('nfe:ICMS/nfe:ICMS61', ns) if imposto is not None else None
                    if icms61 is not None:
                        origem_prod61 = icms61.find('nfe:orig', ns).text if icms61 is not None else None
                        cst_icms61 = icms61.find('nfe:CST', ns).text if icms61 is not None else None
                        vBC_icms61 = icms61.find('nfe:qBCMonoRet', ns).text if icms61 is not None else None
                        pICMS61 = icms61.find('nfe:adRemICMSRet', ns).text if icms61 is not None else None
                        vICMS61 = icms61.find('nfe:vICMSMonoRet', ns).text if icms61 is not None else None


                # Capturar os dados do Pis
                cst_pis = vBC_pis = pPIS = vPIS = cst_cofins = vBC_cofins = pCOFINS = vCOFINS = None
                cst_pis_nt = vBC_pis_nt = pPIS_nt = vPIS_nt = cst_cofins_nt = vBC_cofins_nt = pCOFINS_nt = vCOFINS_nt = None

                if imposto is not None:

                    pisaliq = imposto.find('nfe:PIS/nfe:PISAliq', ns) if imposto is not None else None
                    if pisaliq is not None:
                        cst_pis = pisaliq.find('nfe:CST', ns).text if pisaliq is not None else None
                        vBC_pis = pisaliq.find('nfe:vBC',ns).text if pisaliq is not None else None
                        pPIS = pisaliq.find('nfe:pPIS', ns).text if pisaliq is not None else None
                        vPIS = pisaliq.find('nfe:vPIS', ns).text if pisaliq is not None else None

                    Cofinsaliq = imposto.find('nfe:COFINS/nfe:COFINSAliq', ns) if imposto is not None else None
                    if Cofinsaliq is not None:
                        cst_cofins = Cofinsaliq.find('nfe:CST', ns).text if Cofinsaliq is not None else None
                        vBC_cofins = Cofinsaliq.find('nfe:vBC',ns).text if Cofinsaliq is not None else None
                        pCOFINS = Cofinsaliq.find('nfe:pCOFINS', ns).text if Cofinsaliq is not None else None
                        vCOFINS = Cofinsaliq.find('nfe:vCOFINS', ns).text if Cofinsaliq is not None else None

                    pisnt = imposto.find('nfe:PIS/nfe:PISNT', ns) if imposto is not None else None
                    if pisnt is not None:
                        cst_pis_nt = pisnt.find('nfe:CST', ns).text if pisnt is not None else None

                    cofinsnt = imposto.find('nfe:COFINS/nfe:COFINSNT', ns) if imposto is not None else None
                    if cofinsnt is not None:
                        cst_cofins_nt = cofinsnt.find('nfe:CST', ns).text if cofinsnt is not None else None

                # Adicionar o produto como um registro ao dataframes
                dataframes.append({
                    'Nat_Operacao': nat_op,
                    'Chave_Acesso': chave_acesso,
                    'Numero NF': numero_nota,
                    'Data_Emissao': data_emissao,
                    'CNPJ Emitente': cnpj_emitente,
                    'Nome Emitente': emitente_nome,
                    'UF_Emitente': uf_emitente,
                    'CNPJ_Destinatário': cnpj_destinatario,
                    'Nome_Destinatário': destinatario_nome,
                    'UF_Destinatário': uf_destinatario,
                    'Código EAN': cod_ean,
                    'Nome Produto': nome_produto,
                    'NCM': ncm,
                    'CFOP': cfop,
                    'CEST': cest,
                    'Origem Produto': origem_prod,
                    'CST-ICMS': cst_icms,
                    'Vlr_Produto': vprod,
                    'BC-ICMS': vBC_icms,
                    'Per_ICMS': pICMS,
                    'vlr_ICMS': vICMS,
                    'Orig_Prod10': origem_prod10,
                    'CST_ICMS10': cst_icms10,
                    'BC-ICMS10': vBC_icms10,
                    'Per_ICMS10': pICMS10,
                    'vlr_ICMS10': vICMS10,
                    'BC_ICMS-ST': vBC_icms_ST,
                    'Per_ICMS-ST': pICMS_ST,
                    'Vlr_ICMS-ST': vICMS_ST,
                    'Orig_Prod61': origem_prod61,
                    'BC-ICMS61': vBC_icms61,
                    'Per_ICMS61': pICMS61,
                    'vlr_ICMS61': vICMS61,
                    'CST_PIS': cst_pis,
                    'BC_Pis': vBC_pis,
                    'Per_Pis': pPIS,
                    'Vlr_Pis': vPIS,
                    'CST_COFINS': cst_cofins,
                    'BC_Cofins': vBC_cofins,
                    'Per_Cofins': pCOFINS,
                    'Vlr_Cofins': vCOFINS,
                    'CST_PIS_NT': cst_pis_nt,
                    'CST_COFINS_NT': cst_cofins_nt
                })

# Criar um DataFrame com os dados de todos os arquivos
df = pd.DataFrame(dataframes)


In [ ]:
# Amostra dos dados
df.sample(10)

In [ ]:
df.isnull().sum()

In [ ]:
df[df['CNPJ Emitente'].isna()]

In [ ]:
# Verificando os nomes das colunas
df.columns

In [ ]:
# Tamanho do dataset.
df.shape

In [ ]:
# Converter colunas numéricas
cols_num_float = ['BC-ICMS','BC-ICMS61','vlr_ICMS61' ,'vlr_ICMS','BC-ICMS10', 'vlr_ICMS10', 'BC_ICMS-ST','Vlr_ICMS-ST','Vlr_Produto','BC_Pis', 'Per_Pis','Vlr_Pis','BC_Cofins',
                  'Per_Cofins', 'Vlr_Cofins']
df[cols_num_float]= df[cols_num_float].apply(lambda col: pd.to_numeric(col, errors='coerce', downcast='float').round(2))

In [ ]:
cols_int = ['Per_ICMS','Per_ICMS10','Per_ICMS-ST']
df[cols_int] = df[cols_int].apply(pd.to_numeric, errors='coerce', downcast='integer')

In [ ]:
# Converter coluna de data
df['Data_Emissao'] = pd.to_datetime(df['Data_Emissao'], utc=True).dt.strftime('%Y-%m-%d')

# Na conversão anterior a coluna continuará sendo string, portanto precisamos fazer a conversão novamente, porém sem dt.strftime()
df['Data_Emissao'] = pd.to_datetime(df['Data_Emissao'])

In [ ]:
# Verificando o formato da data.
df.head()

In [ ]:
# Informaçõs do dataset
df.info()

In [ ]:
# o DataFrame apresenta valores nulos ou ausentes apenas nas colunas numéricas, desta forma preencheremos os mesmos com "0"
df = df.fillna(0)

In [ ]:
# Verificando se há valores nulos ainda
df.isnull().sum()

In [ ]:
# Amostra dos dados após conversão dos valores nulos
df.head()

In [ ]:
# Inserindo novas colunas para Base de cálculo do ICMS e Valor do ICMS, concatenando as colunas com ICMS61 e ICMS
df['BC_ICMS'] = df['BC-ICMS'] + df['BC-ICMS61']
df['VLR_ICMS'] = df['vlr_ICMS'] + df['vlr_ICMS61']

# Calcular um valor na coluna BC_Pis e Vlr_Pis subtraindo o valor da coluna VLR_ICMS da coluna bc_icms
df['BC_PIS_Calc'] = df['Vlr_Produto'] - df['VLR_ICMS']
df['VLR_PIS_Calc'] = df['BC_PIS_Calc'] * 0.0165

df['BC_Cofins_Calc'] = df['Vlr_Produto'] - df['VLR_ICMS']
df['VLR_Cofins_Calc'] = df['BC_Cofins_Calc'] * 0.076

In [ ]:
# Exportar para CSV
df.to_excel('notas_fiscais_fornecedores.xlsx')

In [ ]:
#%watermark --iversions

## Dashboard


In [ ]:
#!pip install -q dash

In [ ]:
!pip install pyngrok

In [ ]:
!pip install -q flask

In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc
import pandas as pd
import plotly.express as px
import pyngrok
from pyngrok import ngrok
from flask import Flask


# Incorporate data
#df = pd.read_excel('notas_fiscais_fornecedores.xlsx', sep=';')
df['Data_Emissao'] = pd.to_datetime(df['Data_Emissao'])
df['ano'] = df['Data_Emissao'].dt.year
agrupado = df[['ano','BC_ICMS-ST']].groupby('ano').sum().reset_index()
tabela1 = df[['Numero NF', 'Nome Emitente','Nome Produto', 'NCM', 'Vlr_Produto']]
# Initialize the app
server = Flask(__name__)
app = Dash(__name__, server=server)

# App layout
app.layout = [
    html.Div(children='My First App with Data and a Graph'),
    dash_table.DataTable(style_data={'lineHeight': '15px'},
                         data=tabela1.to_dict('records'),
                         page_size=5,columns=[{'id': c, 'name': c} for c in tabela1.columns]),
    dcc.Graph(figure=px.bar(agrupado, x='ano', y='BC_ICMS-ST'))
]


# Configurar o ngrok
public_url = ngrok.connect(8050)  # Porta do Dash
print("URL público:", public_url)

# Run the app
if __name__ == '__main__':
    #app.run(debug=True),
    app.run_server(debug=True, port=8050)  # Porta alterada para 8051

In [ ]:
URL público: http://<algum-link>.ngrok.io
